## Computitional Social Science - Assignment 2
## Name: Daniyal Saleem
## Matriculation Number: 219203408


# Task 1.1
- Collect the name of all the countries per continent from English Wikipedia

In [ ]:
import requests
page = requests.get("https://simple.wikipedia.org/wiki/List_of_countries_by_continents")
from bs4 import BeautifulSoup #library for parsing HTML code
soup = BeautifulSoup(page.content, 'html.parser') 
# out = open("wiki_Countries.html", "w", encoding="utf-8")
# out.write(str(soup))
# out.close()

## Read in the HTML file
# soup = BeautifulSoup(open("wiki_Countries.html", encoding='utf-8'), "html.parser")

liContinents = soup.find_all("li",{"class":"toclevel-1"})  # getting all li of continetns
# print(liContinents[:7])

continents_list=[]
continents_list_byContinent=[]

for item in liContinents[:7]: #only traversing starting 7 items i.e  only continents
    q=item.find("a") # getting first anchor tag of each li
    href=q.get("href") # getting href attribute  of each anchor tag
    continents_list.append(href[1:])
# print(continents_list)

#For every continent get list of countries
all_uls = soup.find("div",{"class":"mw-parser-output"}).find_all("ul", recursive=False)
# print(all_uls)
for ul in all_uls[:7]: #each individual ul contains list of countries of single continent
    countries_list=[]  #populating countries of individual continent 
    for li in ul.find_all('li', recursive=False):
        a = li.find('a') # getting anchor tag of individual country line item
        if(a):
            country_name = a["title"]
        countries_list.append(country_name)
    continents_list_byContinent.append(countries_list)

continents_list_byContinent.insert(1, []) # adjusting list because ther is no country list (i.e. no ul) in Antarctica

# merging last two lists because there are two lists (i.e. two ul;s) in Oceania/Australia continent
continents_list_byContinent[-2]= continents_list_byContinent[-2] + continents_list_byContinent[-1] 

del continents_list_byContinent[-1] # deleting extra last list after merging
# print(continents_list_byContinent)

dict_countries_by_Continents = {}
for i,item in enumerate(continents_list):
    dict_countries_by_Continents[continents_list[i]] = continents_list_byContinent[i]

dict_countries_by_Continents


# Task 1.2
- Create countries-continents dataframe, this dataframe should have two columns: country , continent.

In [ ]:
import pandas as pd
df = pd.DataFrame(columns=['Country','Continent'])
for continent in dict_countries_by_Continents.keys():
    for country in dict_countries_by_Continents[continent]:
        df = df.append({
             "Country":  country,
             "Continent": continent
              }, ignore_index=True)

df.rename(columns={'Country': 'Country_or_region'}, inplace=True)
df
# df.to_csv('x.csv', sep=',')


# Task 1.3
- Collect the happiness score, GDP per capital, social support, healthy
life expectancy, freedom to make life choices, generosity, and perceptions of corruption per country in 2019 from English Wikipedia and
put all collected information in a dataframe .

In [ ]:
page2 = requests.get("https://en.wikipedia.org/wiki/World_Happiness_Report#2019_report")
soup2 = BeautifulSoup(page2.content, 'html.parser') 

table = soup2.find("table",{"class":"wikitable"}) #getting the main table (2019 Report)
df2 = pd.read_html(str(table))[0] # Pandas method to convert html table to a dataframe:
df2.columns = [c.replace(' ', '_') for c in df2.columns]
df2

# table = soup.find("table",{"class":"wikitable"}) #getting the main table
# table_rows = table.find_all('tr')

# l []
# for tr in table_rows:
# #     print(tr)
#     td = tr.find_all('td')
#     row = [Cell.text for Cell in td]
#     l.append(row)
# pd.DataFrame(l, columns=["Rank","Country", "Happiness Score", "GDP per Capital", "Social Support", "Healthy Life Expectancy", "Freedom to make Life Choices", "Generosity","Perceptions of Corruption"])

# Task 1.4
- Create a new dataframe with all the information that you collected
and save it in a CSV file.

In [ ]:
# result = df.append(df2, sort=False)

result=pd.merge(df, df2, on='Country_or_region')
result=result.sort_values(by=['Overall_rank']) # sort by Hapiness Score

result.to_csv('All_Info.csv', sep=',')
result

# Task 2.1
- Plot the distribution of happiness score per country.

In [ ]:
import matplotlib.pyplot as plt
data=result
Countries =data['Country_or_region']
H_Score = data['Score']

plt.figure(figsize=(12,35))

plt.barh(Countries, H_Score,height=0.4,align='edge')
plt.ylabel("Countries")
plt.xlabel("Happiness Score")
plt.title("Distribution of Happiness Score Per Country.")
    
plt.margins(y=0.005)
plt.show()


# Task 2.2
- Find the top 10 happiest countries.

In [ ]:
data.sort_values(by=['Score']) # sort by Hapiness Score
data.head(10) # Display top 10 Happiest Countries

# Task 2.3
- What is the average of happiness.

In [ ]:
# getting the average value of Happiness Score Column
avg_Happiness = data["Score"].mean() 
print("The average of Happiness is" , avg_Happiness)

# Task 2.4
- What are the countries below the median

In [ ]:
median_Happiness = data["Score"].median() # getting the median value of Happiness Score Column

# Create variable with TRUE if Score is less than the Median of Score
less_than_median = data['Score'] < median_Happiness

#Selecting all row where Happiness Score is less than the Median of Score
data[less_than_median]['Country_or_region']

# Task 2.5
- Plot the correlation between different variables and give an interpretation. 
- For each continent, give the happiest country

In [ ]:
# correlation between Score and GDP_per_capita

x1=data["Score"]
y1=data["GDP_per_capita"]

plt.scatter(x1, y1)
plt.title("correlation between GDP_per_capita and Happiness Score.")
plt.ylabel("GDP_per_capita")
plt.xlabel("Happiness Score")
plt.show()

print("Interpretation: There is weak positive correlation between Score and GDP_per_capita variables.") 
print("It means that one variable will increase if we increase the other variable but the the relationship is not very strong.")


print(" \nrefference: https://www.verywellmind.com/what-is-correlation-2794986")

In [ ]:
#For each continent, give the happiest country

# max happines score of country by continent 
happiest_Country = data.groupby(['Continent'])['Score'].transform(max) == data['Score']
data[happiest_Country]